<div style="display: flex; gap: 10px;">
  <img src="../images/HOOPS_AI.jpg" style="width: 20%;">

# HOOPS AI: EXPERIMENTAL - Flow Trainer

in this notebook we introduce the possibility to just generated the dgl files fo training in the Flow pipeline.
We advice the reader to see the notebook 'ETL_pipeline_suing_flow_fabwave' for a complete study.

when your dataset is ready, and you do not longer need to inspect the files and you have the collection fo cad files that you indeed would liek to train,

then, use this example to generated only the dgl files, and set the DatasetLoader properly.

The Training is perform directly on the dgl files. 

In [1]:
import hoops_ai
import os

hoops_ai.set_license(os.getenv("HOOPS_AI_LICENSE"), validate= False)

HOOPS AI version :  1.0.0-b2dev8 



In [2]:
import pathlib

flows_outputdir = pathlib.Path.cwd().joinpath("out")

## Prepared the ml input files for ml training

this will only create the graph files (.dgl) for training the GraphClassification model

This is optimized for Trianing once you knw what data you need to extract.

In [3]:
# Import task functions from external module for ProcessPoolExecutor compatibility
from scripts.cad_tasks_fabwave import gather_fabwave_files, encode_data_for_ml_training, get_flow_name
flow_name = get_flow_name()

print(f"Flow name: {flow_name}")

2026-01-26 18:07:10 | INFO | numexpr.utils | NumExpr defaulting to 16 threads.


HOOPS AI version :  1.0.0-b2dev8 

Flow name: ETL_Fabwave_training_b2


In [4]:

# data source
datasources_dir = [str(pathlib.Path.cwd().parent.joinpath("packages","cadfiles","fabwave"))]

# Create and run the Data Flow
cad_flow = hoops_ai.create_flow(
    name=flow_name,
    tasks=[gather_fabwave_files, encode_data_for_ml_training],
    max_workers=4,
    #debug=True,
    flows_outputdir=str(flows_outputdir),
    ml_task="Part Classification",
    storage_type = "memory",
    #parallel_task_kwargs=parallel_task_kwargs,
    export_visualization=False  # Disable visualization export
)

# Run the flow to process all files
print("Starting flow execution with parallel processing...")
flow_output, output_dict, flow_file = cad_flow.process(inputs={'cad_datasources': datasources_dir}, clean_ouput_dir = False)

Starting flow execution with parallel processing...
|INFO| FLOW | ######### Flow 'ETL_Fabwave_training_b2' start #######
|INFO| FLOW | 
Flow Execution Summary
|INFO| FLOW | ==================================================
|INFO| FLOW | Task 1: gather fabwave files
|INFO| FLOW |     Inputs : cad_datasources
|INFO| FLOW |     Outputs: cad_dataset
|INFO| FLOW | Task 2: Preparing data for Exploring and ML training
|INFO| FLOW |     Inputs : cad_dataset
|INFO| FLOW |     Outputs: cad_files_encoded
|INFO| FLOW | 
Task Dependencies:
|INFO| FLOW | gather fabwave files has no dependencies.
|INFO| FLOW | gather fabwave files --> Preparing data for Exploring and ML training
|INFO| FLOW | ==================================================

|INFO| FLOW | Executing ParallelTask 'gather fabwave files' with 1 items.


C:\Users\LuisSalazar\MAIN\repos\ML-Initiative\src\hoops_ai\flowmanager\flow_builder.py:377: UserWarning: auto_dataset_export automatically disabled for memory storage type. Memory storage cannot be merged into datasets.
  warnings.warn(


DATA INGESTION:   0%|                                                                            | 0/1 [00:00<…

|INFO| FLOW | Executing ParallelTask 'Preparing data for Exploring and ML training' with 4572 items.


DATA TRANSFORMATION:   0%|                                                                    | 0/4572 [00:00<…

|INFO| FLOW | Time taken: 673.28 seconds
|INFO| FLOW | ######### Flow 'ETL_Fabwave_training_b2' end ######


KeyboardInterrupt: 

## Define the data split and training conditions

In [5]:
from hoops_ai.dataset import DatasetLoader 
# read the dgl files form the flow output and inject it to the dataset loader
#dgl files
dgl_dir = flows_outputdir.joinpath("flows",flow_name, "dgl")
print(f"DGL files directory: {dgl_dir}")
# get all .ml file in that folder a list
dgl_files = [str(f) for f in dgl_dir.glob("*.ml")]
print(f"Number of DGL files generated: {len(dgl_files)}")


cadflowdataset = DatasetLoader(graph_files=dgl_files, file_code_start=1)

cadflowdataset.split(key="random", train=0.8, validation=0.1, test=0.1)

DGL files directory: C:\Users\LuisSalazar\MAIN\repos\HOOPS-AI-tutorials\notebooks\out\flows\ETL_Fabwave_training_b2\dgl
Number of DGL files generated: 4546


(3637, 455, 454)

## Train GraphClassification model

In [6]:
from hoops_ai.ml.EXPERIMENTAL import GraphClassification, FlowTrainer
# Define the Graph Classification model

PartClassification_HOOPS = GraphClassification(num_classes=45,
                                               result_dir = flows_outputdir)

flow_trainer_HOOPS = FlowTrainer(

    flowmodel       = PartClassification_HOOPS,
    datasetLoader   = cadflowdataset,
    experiment_name = "HOOPS_AI_train",
    result_dir      = flows_outputdir,
    accelerator     = 'cpu',
    devices         = 1, #[0]
    max_epochs      = 1,
    batch_size      = 32  # Use 32, 64 or 128 depending on your avaiable memory
)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


GRAPH CLASSIFICATION - HOOPS GAT - MODEL


In [ ]:
#corrupted_items = flow_trainer_HOOPS.purify()
#print("Removing corrupted items :", len(corrupted_items))

In [7]:
trained_HOOPS_model = flow_trainer_HOOPS.train()
print(f"Training finished. Model checkpoint saved in {trained_HOOPS_model}")



-----------------------------------------------------------------------------------
GRAPH CLASSIFICATION - HOOPS GAT - MODEL - TRAINING STEP
-----------------------------------------------------------------------------------
Training batch size               : 32
Adjusted learning rate (for batch): 0.002

Train set contains                : 3637 samples (80.00%)
Validation set contains           : 455 samples (10.01%)
Test set contains                 : 454 samples (9.99%)
Total samples                     : 4546
Max Epoch                         : 1

The trained model: C:\Users\LuisSalazar\MAIN\repos\HOOPS-AI-tutorials\notebooks\out\ml_output\HOOPS_AI_train\0126\181902\best.ckpt

To monitor the logs, run:
tensorboard --logdir results/HOOPS_AI_train/0126/181902
-----------------------------------------------------------------------------------
        


Training: 0it [00:00, ?it/s]

C:\Users\LuisSalazar\miniconda3\envs\hoops_ai_cpu_dev\lib\site-packages\pytorch_lightning\trainer\trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Training finished. Model checkpoint saved in C:\Users\LuisSalazar\MAIN\repos\HOOPS-AI-tutorials\notebooks\out\ml_output\HOOPS_AI_train\0126\181902\best.ckpt

-----------------------------------------------------------------------------------
GRAPH CLASSIFICATION - HOOPS GAT - MODEL - TESTING STEP
-----------------------------------------------------------------------------------

The trained model: C:\Users\LuisSalazar\MAIN\repos\HOOPS-AI-tutorials\notebooks\out\ml_output\HOOPS_AI_train\0126\181902\best.ckpt

Test set contains 454 training samples
-----------------------------------------------------------------------------------
        


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/LuisSalazar/MAIN/repos/HOOPS-AI-tutorials/notebooks/out/ml_output/HOOPS_AI_train/0126/181902/best.ckpt'

In [8]:
flow_trainer_HOOPS.test(trained_HOOPS_model)
print(f"Testing finished")


-----------------------------------------------------------------------------------
GRAPH CLASSIFICATION - HOOPS GAT - MODEL - TESTING STEP
-----------------------------------------------------------------------------------

The trained model: C:\Users\LuisSalazar\MAIN\repos\HOOPS-AI-tutorials\notebooks\out\ml_output\HOOPS_AI_train\0126\181902\best.ckpt

Test set contains 454 training samples
-----------------------------------------------------------------------------------
        


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/LuisSalazar/MAIN/repos/HOOPS-AI-tutorials/notebooks/out/ml_output/HOOPS_AI_train/0126/181902/best.ckpt'